In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Spoiler Detection/LM-BFF"
%ls

/content/drive/MyDrive/Spoiler Detection/LM-BFF
 auto_label_mapping/               log.lock
 auto_template/                    README.md
 data/                             requirements.txt
 discord_log                       result/
 eda.ipynb                         run_experiment.sh
 ensemble_predict_results/         run.ipynb
'ensemble_predict_results'$'\r'/   run_multiple.sh
 ensemble.sh                       run.py
 env/                              spoilers_auto_label_mapping/
 evaluate.ipynb                    spoilers_auto_template/
 figs/                             src/
 label_search.sh                   template_search.sh
 LICENSE                           tools/
 log


In [3]:
!source env/bin/activate; python -m pip install --upgrade pip

In [ ]:
!apt-get install vim

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgpm2 vim-common vim-runtime xxd
Suggested packages:
  gpm ctags vim-doc vim-scripts
The following NEW packages will be installed:
  libgpm2 vim vim-common vim-runtime xxd
0 upgraded, 5 newly installed, 0 to remove and 39 not upgraded.
Need to get 6,725 kB of archives.
After this operation, 32.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 xxd amd64 2:8.0.1453-1ubuntu1.8 [49.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 vim-common all 2:8.0.1453-1ubuntu1.8 [71.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgpm2 amd64 1.20.7-5 [15.1 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 vim-runtime all 2:8.0.1453-1ubuntu1.8 [5,435 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 vim amd64 2:8.0.1453

In [ ]:
!nvidia-smi

Sun Apr 17 05:48:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!source env/bin/activate; python tools/generate_k_shot_data.py --task spoilers --k 250

K = 250
Seed = 100
| Task = spoilers
Seed = 13
| Task = spoilers
Seed = 21
| Task = spoilers
Seed = 42
| Task = spoilers
Seed = 87
| Task = spoilers


In [20]:
!source env/bin/activate; TAG=no_demo TYPE=prompt TASK=spoilers BS=4 LR=2e-5 SEED=21 MODEL=roberta-large bash run_experiment.sh \
 "--template *cls**sent_0*._Relevant?*mask*.*sep+* --mapping {0:'Yes',1:'No'} --first_sent_limit 502 --other_sent_limit 502"

/content/drive/MyDrive/Spoiler Detection/LM-BFF/env/lib/python3.7/site-packages/transformers/training_args.py:339: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
04/18/2022 09:31:05 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
04/18/2022 09:31:05 - INFO - __main__ -   Training/evaluation parameters DynamicTrainingArguments(output_dir='result/no_demo-spoilers-prompt-250-21-roberta-large', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_

In [14]:
!source env/bin/activate; TAG=irrelevant TYPE=prompt-demo TASK=spoilers BS=4 LR=2e-5 SEED=21 MODEL=roberta-large bash run_experiment.sh \
 "--template *cls**sent_0*._Irrelevant?*mask*.*sep+* --mapping {0:'No',1:'Yes'} --first_sent_limit 163 --other_sent_limit 163"

/content/drive/MyDrive/Spoiler Detection/LM-BFF/env/lib/python3.7/site-packages/transformers/training_args.py:339: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
04/18/2022 06:01:48 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
04/18/2022 06:01:48 - INFO - __main__ -   Training/evaluation parameters DynamicTrainingArguments(output_dir='result/irrelevant-spoilers-prompt-demo-250-21-roberta-large', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=

In [ ]:
import numpy as np
test = np.load('result/relevant-spoilers-prompt-demo-250-21-roberta-large/spoilers--1--1.npy')

In [ ]:
test

array([[48.384468, 45.80882 ],
       [44.264988, 46.22074 ],
       [50.94467 , 45.77304 ],
       ...,
       [50.57451 , 46.24546 ],
       [47.357555, 45.11906 ],
       [46.39147 , 46.88502 ]], dtype=float32)

In [ ]:

!source env/bin/activate; bash run_multiple.sh

Streaming output truncated to the last 5000 lines.
2220it [15:34,  4.33it/s]
2221it [15:34,  4.33it/s]
2222it [15:34,  4.33it/s]
2223it [15:35,  4.33it/s]
2224it [15:35,  4.34it/s]
2225it [15:35,  4.34it/s]
2226it [15:35,  4.33it/s]
2227it [15:35,  4.33it/s]
2228it [15:36,  4.34it/s]
2229it [15:36,  4.33it/s]
2230it [15:36,  4.33it/s]
2231it [15:36,  4.33it/s]
2232it [15:37,  4.33it/s]
2233it [15:37,  4.33it/s]
2234it [15:37,  4.33it/s]
2235it [15:37,  4.34it/s]
2236it [15:38,  4.33it/s]
2237it [15:38,  4.33it/s]
2238it [15:38,  4.33it/s]
2239it [15:38,  4.33it/s]
2240it [15:38,  4.33it/s]
2241it [15:39,  4.33it/s]
2242it [15:39,  4.33it/s]
2243it [15:39,  4.33it/s]
2244it [15:39,  4.33it/s]
2245it [15:40,  4.34it/s]
2246it [15:40,  4.34it/s]
2247it [15:40,  4.34it/s]
2248it [15:40,  4.34it/s]
2249it [15:41,  4.34it/s]
2250it [15:41,  4.34it/s]
2251it [15:41,  4.34it/s]
2252it [15:41,  4.33it/s]
2253it [15:41,  4.33it/s]
2254it [15:42,  4.33it/s]
2255it [15:42,  4.33it/s]
2256it [15:42

In [19]:
#yes_no: 89.5 (3.3) -> TEMPLATE="*cls**sent_0*._Spoiler?*mask*.*sep+*" MAPPING="{0:'No',1:'Yes'}"
!source env/bin/activate; python tools/gather_result.py --condition "{'tag': 'relevant', 'task_name': 'spoilers', 'few_shot_type': 'prompt-demo'}"

21-21: best dev (0.85511) test (0.87201) test2 (0.79139) | total trials: 2
    | gradient_accumulation_steps: 1 | learning_rate: 2e-05 | per_device_train_batch_size: 4 | eval_steps: 100 | max_steps: 1000 
mean +- std: 87.2 (0.0) (median 87.2)second metric: 79.1 (0.0) (median 79.1)


In [ ]:
!source env/bin/activate; bash tools/get_sbert_embedding.sh roberta-large

spoilers-16-42-train
0it [00:00, ?it/s]| GABI USING HACKS
32it [00:00, 41.33it/s]
spoilers-16-42-dev
0it [00:00, ?it/s]| NO
185it [00:04, 42.41it/s]
spoilers-16-13-train
0it [00:00, ?it/s]| its Joe titan
32it [00:00, 42.19it/s]
spoilers-16-13-dev
0it [00:00, ?it/s]| Cart titan X ???
185it [00:04, 42.07it/s]
spoilers-16-21-train
0it [00:00, ?it/s]| gabi sniper ez
32it [00:00, 41.84it/s]
spoilers-16-21-dev
0it [00:00, ?it/s]| Guys, why goth mikasa was shown?
185it [00:04, 43.03it/s]
spoilers-16-87-train
0it [00:00, ?it/s]| Well for manga readers like me
32it [00:00, 42.66it/s]
spoilers-16-87-dev
0it [00:00, ?it/s]| Right?
185it [00:04, 42.70it/s]
spoilers-16-100-train
0it [00:00, ?it/s]| got monkey fall in this ep
32it [00:00, 42.71it/s]
spoilers-16-100-dev
0it [00:00, ?it/s]| if only the beast titan animation was at a higher frame rate
185it [00:04, 42.99it/s]
spoilers-16-42-train
0it [00:00, ?it/s]| GABI USING HACKS
32it [00:00, 41.09it/s]
spoilers-16-42-dev
0it [00:00, ?it/s]| NO
185i

In [ ]:
!source env/bin/activate; TAG=handcrafted_10x TYPE=prompt-demo TASK=spoilers BS=8 LR=2e-5 SEED=42 MODEL=roberta-large bash run_experiment.sh "--demo_filter --demo_filter_model sbert-roberta-large"

/content/drive/MyDrive/Spoiler Detection/LM-BFF/env/lib/python3.7/site-packages/transformers/training_args.py:339: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
03/01/2022 12:09:47 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
03/01/2022 12:09:47 - INFO - __main__ -   Training/evaluation parameters DynamicTrainingArguments(output_dir='result/spoilers-prompt-demo-16-42-roberta-large-28525', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam

In [ ]:
!source env/bin/activate; python tools/generate_template.py \
    --output_dir spoilers_auto_template \
    --task_name spoilers \
    --seed 13 21 42 87 100 \
    --t5_model t5-3b \
    --beam 100

Downloading: 100% 1.20k/1.20k [00:00<00:00, 1.15MB/s]
Downloading: 100% 11.4G/11.4G [04:55<00:00, 38.6MB/s]
Downloading: 100% 792k/792k [00:00<00:00, 11.1MB/s]
# spoilers 16 13 100
| dataset examples
| {'label': 0, 'text': ['fuck him']}
| {'label': 1, 'text': ['AAAAAAA eyy levi vs zeke alrdy happening djnjkfbje,rkfker im not ready']}

| mapping
| {0: 'No', 1: 'Yes'}
####### example #######
Fuck him<extra_id_0> No<extra_id_1>
 ⁇ <extra_id_0> No<extra_id_1>
LES GOO<extra_id_0> No<extra_id_1>
####### example #######

100% 18/18 [17:22<00:00, 57.90s/it]
####### generated results #######
--------------
score: -1.2218170166015625
generated ids [32099, 5, 32098, 5, 1]
generated text <extra_id_0>.<extra_id_1>.</s>
--------------
score: -2.7679529190063477
generated ids [32099, 5, 32098, 55, 1]
generated text <extra_id_0>.<extra_id_1>!</s>
--------------
score: -2.8754196166992188
generated ids [32099, 58, 32098, 5, 1]
generated text <extra_id_0>?<extra_id_1>.</s>
--------------
score: -3.01939

In [ ]:
!source env/bin/activate; bash template_search.sh

Streaming output truncated to the last 5000 lines.
43it [02:37,  1.99s/it]
44it [02:37,  1.46s/it]
45it [02:37,  1.09s/it]
46it [02:37,  1.20it/s]
47it [02:38,  1.53it/s]
Epoch:  50% 124/250 [04:00<03:28,  1.65s/it]
49it [03:19, 12.83s/it]
50it [03:20,  9.05s/it]
51it [03:20,  6.40s/it]
52it [03:20,  4.55s/it]
53it [03:20,  3.26s/it]
54it [03:21,  2.35s/it]
55it [03:21,  1.71s/it]
56it [03:21,  1.27s/it]
57it [03:21,  1.05it/s]
58it [03:21,  1.35it/s]
59it [03:22,  1.70it/s]
60it [03:22,  2.23it/s]03/07/2022 04:08:09 - INFO - src.trainer -   Best dev result: 0.9296250343322754
Epoch:  60% 149/250 [04:51<02:49,  1.68s/it]
61it [04:11, 15.07s/it]
62it [04:11, 10.62s/it]
63it [04:11,  7.50s/it]
64it [04:12,  5.32s/it]
65it [04:12,  3.79s/it]
66it [04:12,  2.73s/it]
67it [04:12,  1.98s/it]
68it [04:13,  1.45s/it]
69it [04:13,  1.09s/it]
70it [04:13,  1.20it/s]
71it [04:13,  1.54it/s]
Epoch:  70% 174/250 [05:35<02:05,  1.66s/it]
73it [04:55, 12.82s/it]
74it [04:55,  9.05s/it]
75it [04:55,  

In [ ]:
!source env/bin/activate; python tools/sort_template.py --condition "{'tag': 'yes-no-template21', 'task_name': 'spoilers'}" --template_dir spoilers_auto_template --name yes_no

Seed 21 has 100 results


In [ ]:
# !source env/bin/activate; bash tools/run_generate_labels.sh

+ K=16
+ DATA_DIR=data/k-shot-10x
+ OUTPUT_DIR=spoilers_auto_label_mapping
+ MODEL_NAME=roberta-large
+ LOAD_TEMPLATES=true
+ TEMPLATE_DIR=spoilers_auto_template
+ TEMPLATE_NAME=yes_no
+ NUM_TEMPLATES=20
+ K_LIKELY=100
+ K_NEIGHBORS=30
+ N_PAIRS=100
+ TASKS=spoilers
+ SEEDS=21
+ TASK_EXTRA=
+ for TASK in $TASKS
+ for SEED in $SEEDS
+ case $TASK in
+ TEMPLATE='*cls**sent_0*._Spoiler?*mask*.*sep+*'
+ MAPPING='{0:'\''No'\'',1:'\''Yes'\''}'
+ [[ true = \t\r\u\e ]]
+ FILENAME=spoilers_auto_template/yes_no/16-21.sort.txt
++ head -n 20 spoilers_auto_template/yes_no/16-21.sort.txt
+ for TEMPLATE in $(head -n $NUM_TEMPLATES $FILENAME)
+ python tools/generate_labels.py --overwrite_output_dir --output_dir /tmp/output --model_name_or_path roberta-large --output_file spoilers_auto_label_mapping/auto_template/spoilers/16-21.txt --append_output_file --write_template --template '*cls**sent_0*.*mask*s.*sep+*' --mapping '{0:'\''No'\'',1:'\''Yes'\''}' --task_name spoilers --data_dir data/k-shot-10x/spoil

In [ ]:
# !source env/bin/activate; bash label_search.sh

Streaming output truncated to the last 5000 lines.
 50% 6/12 [00:01<00:01,  4.92it/s]
 58% 7/12 [00:01<00:01,  4.73it/s]
 67% 8/12 [00:01<00:00,  4.60it/s]
 75% 9/12 [00:01<00:00,  4.51it/s]
 83% 10/12 [00:02<00:00,  4.45it/s]
 92% 11/12 [00:02<00:00,  4.42it/s]
100% 12/12 [00:02<00:00,  5.09it/s]03/08/2022 05:11:42 - INFO - src.trainer -   Best dev result: 0.9383749961853027
Epoch:  20% 49/250 [01:32<05:35,  1.67s/it]
13it [00:52, 15.14s/it]            
14it [00:52, 10.67s/it]
15it [00:52,  7.54s/it]
16it [00:53,  5.34s/it]
17it [00:53,  3.81s/it]
18it [00:53,  2.74s/it]
19it [00:53,  1.98s/it]
20it [00:54,  1.46s/it]
21it [00:54,  1.09s/it]
22it [00:54,  1.20it/s]
23it [00:54,  1.53it/s]
24it [00:54,  2.02it/s]03/08/2022 05:12:34 - INFO - src.trainer -   Best dev result: 0.9388749599456787
Epoch:  30% 74/250 [02:23<04:52,  1.66s/it]
25it [01:43, 15.03s/it]
26it [01:44, 10.59s/it]
27it [01:44,  7.48s/it]
28it [01:44,  5.31s/it]
29it [01:44,  3.78s/it]
30it [01:44,  2.72s/it]
31it [01:

In [ ]:
!source env/bin/activate; bash ensemble.sh

Streaming output truncated to the last 5000 lines.
2224it [15:36,  4.33it/s]
2225it [15:36,  4.33it/s]
2226it [15:36,  4.33it/s]
2227it [15:36,  4.33it/s]
2228it [15:37,  4.33it/s]
2229it [15:37,  4.33it/s]
2230it [15:37,  4.33it/s]
2231it [15:37,  4.33it/s]
2232it [15:38,  4.33it/s]
2233it [15:38,  4.33it/s]
2234it [15:38,  4.33it/s]
2235it [15:38,  4.33it/s]
2236it [15:39,  4.33it/s]
2237it [15:39,  4.33it/s]
2238it [15:39,  4.33it/s]
2239it [15:39,  4.33it/s]
2240it [15:39,  4.33it/s]
2241it [15:40,  4.33it/s]
2242it [15:40,  4.33it/s]
2243it [15:40,  4.33it/s]
2244it [15:40,  4.33it/s]
2245it [15:41,  4.33it/s]
2246it [15:41,  4.33it/s]
2247it [15:41,  4.33it/s]
2248it [15:41,  4.33it/s]
2249it [15:42,  4.33it/s]
2250it [15:42,  4.33it/s]
2251it [15:42,  4.33it/s]
2252it [15:42,  4.33it/s]
2253it [15:42,  4.33it/s]
2254it [15:43,  4.33it/s]
2255it [15:43,  4.33it/s]
2256it [15:43,  4.33it/s]
2257it [15:43,  4.33it/s]
2258it [15:44,  4.33it/s]
2259it [15:44,  4.33it/s]
2260it [15:44

In [ ]:
!source env/bin/activate; python tools/ensemble.py --condition "{'tag': 'yes_no_ensemble', 'task_name': 'spoilers', 'few_shot_type': 'prompt-demo'}" --n_models 20

Model 0 | val: mean +- std: 94.4 +- 0.0 | test: mean +- std: 88.4 (0.0) (median 88.4) / 95.8 +- 0.0 (median 95.8)
Model 1 | val: mean +- std: 94.4 +- 0.0 | test: mean +- std: 90.3 (0.0) (median 90.3) / 95.8 +- 0.0 (median 95.8)
Model 2 | val: mean +- std: 95.2 +- 0.0 | test: mean +- std: 89.7 (0.0) (median 89.7) / 95.8 +- 0.0 (median 95.8)
Model 3 | val: mean +- std: 88.0 +- 0.0 | test: mean +- std: 83.4 (0.0) (median 83.4) / 91.7 +- 0.0 (median 91.7)
Model 4 | val: mean +- std: 91.6 +- 0.0 | test: mean +- std: 85.1 (0.0) (median 85.1) / 95.8 +- 0.0 (median 95.8)
Model 5 | val: mean +- std: 86.5 +- 0.0 | test: mean +- std: 86.7 (0.0) (median 86.7) / 75.0 +- 0.0 (median 75.0)
Model 6 | val: mean +- std: 87.7 +- 0.0 | test: mean +- std: 84.4 (0.0) (median 84.4) / 100.0 +- 0.0 (median 100.0)
Model 7 | val: mean +- std: 93.1 +- 0.0 | test: mean +- std: 90.2 (0.0) (median 90.2) / 83.3 +- 0.0 (median 83.3)
Model 8 | val: mean +- std: 86.0 +- 0.0 | test: mean +- std: 81.8 (0.0) (median 81.8) 